In [1]:
from Bio import SeqIO
import requests
import sys

# myNEO assessment: Programming challenge

__Assignment__: Decode the given DNA string, using the information available in [_Towards practical, high-capacity, low-maintenance information storage in synthesized DNA_](https://www.researchgate.net/publication/235375794_Towards_practical_high-capacity_low-maintenance_information_storage_in_synthesized_DNA), and answer the resulting question.

## 1. DNA decoding

### 1.1 Huffman encoding

Encoding and decoding require the original base-3 Huffman encoding scheme. The authors published this along with other supplementary data in the [DNA_storage directory](https://www.ebi.ac.uk/goldman-srv/DNA-storage/orig_files/).

In [4]:
#File URL
URL = "https://www.ebi.ac.uk/goldman-srv/DNA-storage/orig_files/View_huff3.cd.new.correct"

#File is stored locally
response = requests.get(URL)
with open('View_huff3.cd.new.correct', "wb") as fh:
    fh.write(response.content)

The file is [ANSI](https://en.wikipedia.org/wiki/Windows-1252) encoded, an extension of the more common [ASCII](https://en.wikipedia.org/wiki/ASCII) format. All characters are saved along with their base-3 code.

In [3]:
#Open file
with open('View_huff3.cd.new.correct', encoding = 'ANSI') as fh:
    #Make dictionary to save base3 encoding
    huffman_dict = {}
    for line in fh:
        line = line.split()
        #Required due to formatting error in original file (line 175)
        if len(line) == 8:
            #Add ANSI character and base3 to dictionary
            huffman_dict[line[3]] = line[1]
        
#Add the encoding for spaces (splitting removed these)
huffman_dict['02212'] = ' '

#Save mirrored dictionary for encoding
ansi_dict = {value:key for key, value in huffman_dict.items()}


These dictionaries can be used to encode and decode information.

### 1.2 Algorithms
Seeing as the unknown string is unfragmented, the following procedures only have to cover up to step 1.4 in [files2features.pdf](https://www.ebi.ac.uk/sites/ebi.ac.uk/files/groups/goldman/file2features_2.0.pdf).

#### 1.2.1 Encoding

Implementing the encoding procedures is not required to complete the challenge, but is performed nonetheless to better understand the algorithm.

The encoding of ANSI text to DNA consists of various steps. For the sake of readability, these steps are split up into individual functions.

In [6]:
#DNA scheme
DNA_dict = {'A':'CGT',
            'C':'GTA',
            'G':'TAC',
            'T':'ACG'}

#Convert integers >=10 to base3
def int_to_base3(n):
    if n == 0:
        return '0'
    nums = []
    while n:
        n, r = divmod(n, 3)
        nums.append(str(r))
        
    return ''.join(reversed(nums))

#Convert ANSI to base3
def ANSI_to_base3(ansi):
    #S1
    S1 = ''
    for i in ansi:        
        S1 += ansi_dict[i]

    #S2
    n = len(S1)
    n_base3 = int_to_base3(n)
    S2 = n_base3.zfill(25)

    #S3
    S3 = ''.zfill(100-n)

    #S4
    S4 = S2 + S1 + S3

    return S4

#Convert base3 to a DNA sequence
def base3_to_DNA(base3_str):

   #Initialize DNA string 
    DNA_str = ''

   #Encode base3_str to DNA
    for i, trit in enumerate(base3_str):
       
       if i == 0:
           next_nt = DNA_dict['A'][int(trit)]

       else:
           previous_nt = DNA_str[-1]
           next_nt = DNA_dict[previous_nt][int(trit)]

       DNA_str += next_nt

    return DNA_str

These functions can be combined to perform the encoding in a single step.

In [31]:
#Encode a given ANSI text
def encode(text):
    #ANSI to base3
    base3 =  ANSI_to_base3(text)

    #Base3 to DNA
    DNA = base3_to_DNA(base3)
    
    return DNA

#### 1.2.2 Decoding

The decoding of DNA to ANSI text consists of various steps. For the sake of readability, these steps are split up into individual functions.

In [28]:
#Convert base3 to integers <= 10
def base3_to_int(num):
    ans = 0
    for c in map(int, num):
        ans = 3 * ans + c
    return ans

#Extract S1 from encoded string
def find_S1(huffman):

    #Find start of S2
    for i, number in enumerate(huffman):
        if number in '12':
            start =  i
            break
    
    #Find S2 (length of S1)
    S2_base3 = huffman[start:25]
    S2 = base3_to_int(S2_base3)

    #Find S1
    S1 = huffman[25:25+S2]

    return S1
    

#Convert DNA to base3
def DNA_to_base3(seq):
    huffman = ''
    for i, current_nt in enumerate(seq):
        if i > 0:
            prev_nt = seq[i-1]
            trit = DNA_dict[prev_nt].find(current_nt)
        else:
            trit = DNA_dict['A'].find(current_nt)

        huffman += str(trit)

    #Return the substring containing the encoded information (S1)
    return find_S1(huffman)


#Convert base3 to ANSI characters
def base3_to_ANSI(base3):
    ansi = ''
    while len(base3) != 0:

        #Check if valid base3 of length 6 occurs
        if base3[:6] in huffman_dict:
            #Decode
            ansi += huffman_dict[base3[:6]]
            #Remove decoded parts
            base3 = base3[6:]

        #Check if valid base3 of length 5 occurs
        elif base3[:5] in huffman_dict:
            #Decode
            ansi += huffman_dict[base3[:5]]
            #Remove decoded parts
            base3 = base3[5:]

        #No valid base3 is found
        else:
            print('Something is wrong, I can feel it!')
            sys.exit()

    return ansi

These functions can be combined to perform the decoding in a single step.

In [29]:
#Decode a given DNA sequence
def decode(DNA):
    #DNA to base3
    base3 =  DNA_to_base3(DNA)

    #Base3 to ANSI
    text = base3_to_ANSI(base3)
    
    return text


__Note__: Not all base3-encoded ANSI characters are equally long; in this case, they can either be 5 or 6 trits long.

The decoding algorithm used to transform base3 into ANSI assumes that characters with longer base3 encoding do not contain other encoded ANSI characters, as this would cause unresolvable ambiguity. 

Example: If __222212__ is valid, __22221__ cannot be a valid encoding key.

This assumption is proven below for this specific encoding scheme.





In [11]:
def proof(dict):
    for key in dict:
        if len(key) == 6:
            if key[:5] in dict:
                return 'The assumption was violated'
    return 'The assumption was not violated'

print(proof(huffman_dict))

The assumption was not violated


### 1.3 Example testing
The authors showcased the algorithm step-by-step in [files2features.pdf](https://www.ebi.ac.uk/sites/ebi.ac.uk/files/groups/goldman/file2features_2.0.pdf).

We will check if our algorithm correctly decodes/encodes the example they provided.


In [32]:
#Define text and correct encoding
example_txt = 'Birney and Goldman'
example_DNA = 'CGTACGTACGTACGTACGTAGTCGCACTACACAGTCGACTACGCTGTACTGCAGAGTGCTGTCTCACGTGATGACGTGCTGCATGATATCTACAGTCATCGTCTATCGAGATACGCTACGTACGT'

#Encode the example txt
encoded = encode(example_txt)

#Check correctness
print(f'The encoding procedure is correct: {encoded == example_DNA}')

#Decode the example DNA
decoded = decode(example_DNA)

print(f'The decoding procedure is correct: {decoded == example_txt}')


The encoding procedure is correct: True
The decoding procedure is correct: True


Both encoding and decoding algorithms work as intended, we can move on on to decoding the unknown DNA sequence. 

### 1.4 myNEO DNA sequence

The given file __MyNeo_dna.txt__ contains a single strand of encoded DNA. 

In [34]:
with open('myNEO_dna.txt') as fh:
    MyNeo_DNA = fh.read()

print(f'Length of DNA sequence: {len(MyNeo_DNA)}')




Length of DNA sequence: 800


Using our algorithm, we can decode the DNA to extract the question.

In [14]:
#Decode unknown sequence
solution = decode(MyNeo_DNA)

#Give solution
print(solution)

Give the number of human swissprot proteins that contain the AA-sequence of the shortest human swissprot protein twice in a window of 100 amino acids![NL]


# 2. Solving the question

The DNA contained the following question: 

_Give the number of human swissprot proteins that contain the AA-sequence of the shortest human swissprot protein twice in a window of 100 amino acids!_

We can split this question into a number of smaller tasks.

1. Obtain AA sequences of all human SwissProt proteins
2. Find the protein with the shortest AA-sequence
3. Look for the pattern in each protein

### 2.1 Obtain AA-sequences of all human SwissProt proteins

We download a FASTA file containing information on all human proteins present in [SwissProt](https://www.uniprot.org/uniprot/?query=reviewed:yes).

In [16]:
#Obtain URL containing the FASTA file
URL = 'https://www.uniprot.org/uniprot/?query=*&format=fasta&fil=organism:%22Homo%20sapiens%20(Human)%20[9606]%22%20AND%20reviewed:yes'
response = requests.get(URL)

#Write to FASTA file
with open("human_proteome.fna", "wb") as fh:
    fh.write(response.content)


__human_proteome.fna__ now contains FASTA-formatted information on every manually annotated and reviewed human protein available.

### 2.2 Find shortest protein in the human proteome

In [36]:
#Initiate search with excessively long fake protein
shortest_p_len = 1000
shortest_p_name = 'Not an actual protein'
shortest_p_seq = 'Not an actual sequence'

#Open file
for fasta in SeqIO.parse(open('human_proteome.fna'),'fasta'):

    #Extract name and sequence
    name, sequence = fasta.id, str(fasta.seq)

    #Check if AA-sequence is shorter than the current shortest sequence
    if len(sequence) < shortest_p_len:

        #Update information
        shortest_p_name = name
        shortest_p_seq = sequence
        shortest_p_len = len(sequence)


#Return information on shortest protein
print(f'{shortest_p_name} is the shortest human protein.')
print(f'Amino acid sequence: {shortest_p_seq} (length = {shortest_p_len})')

sp|P0DPR3|TRDD1_HUMAN is the shortest human protein.
Amino acid sequence: EI (length = 2)


The shortest human protein is called P0DPR3, and is only 2 amino acids long (Glutamic acid - Isoleucine). We can now check which human proteins contain this sequence at least twice in a span of 100 amino acids.

### 2.3 Scan each protein to find the pattern

In [37]:
#Define pattern and initialize output list
pattern = shortest_p_seq
matches = []

#Function to determine if a pattern occurs at least twice in a given sequence
def valid_protein(seq):
    if seq.count(pattern) >= 2:
        return True
    return False

#Open file
for fasta in SeqIO.parse(open('human_proteome.fna'),'fasta'):

    #Extract name and sequence
    name, sequence = fasta.id, str(fasta.seq)

    #Shorter than 100 AA
    if len(sequence) <= 100:
        
        #Search pattern
        if valid_protein(sequence):
            matches.append(name)

    #Longer than 100 AA
    else:
        #Sliding window of 100 AA long
        for i in range(len(sequence)-99):

            #Current window
            window = sequence[i:i+100]

            #Search pattern
            if valid_protein(window):
                
                #Exit loop if protein is valid (avoids continuously checking the same valid protein)
                matches.append(name)
                break

            
#Conclusion
solution = len(matches)
print(solution)

5101


### Conclusion

In total, 5101 proteins fulfill the requirements set by the question.